## **1. Find the corresponding positive values for NER, POS, Chunk tags**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **2. Data Preprocessing for BERT Model (Apply Hugging Face Data)**

### (1) Hugging Face Dataset Conll2003 Exploration

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
# dataset = load_dataset('conll2003')

### (2) Covert Data to BERT Input Style

In [ ]:
!pip install transformers seqeval[gpu]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig #BertForTokenClassification

In [ ]:
MAX_LEN = 128     
TRAIN_BATCH_SIZE = 4
TEST_BATCH_SIZE = 2
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/NLP/Final Project/New Dataset/Bank/ner_datasetreference_new.csv", encoding='unicode_escape')
data.head()

data.count()

Sentence #      47959
Word          1048575
POS           1048575
Tag           1048575
dtype: int64

In [ ]:
'''
step 2a: process NE tags and POS tags
'''
# NE 
"""There are 8 category tags, each with a "beginning" and "inside" variant, and the "outside" tag. It is not really clear what these tags mean - "geo" probably stands for geographical entity, "gpe" for geopolitical entity, and so on. They do not seem to correspond with what the publisher says on Kaggle. Some tags seem to be underrepresented. Let's print them by frequency (highest to lowest): """

# tags = {}
# for tag, count in zip(frequencies_NE.index, frequencies_NE):
#     if tag != "O":
#         if tag[2:5] not in tags.keys():
#             tags[tag[2:5]] = count
#         else:
#             tags[tag[2:5]] += count
#     continue

# print(sorted(tags.items(), key=lambda x: x[1], reverse=True))

"""Let's remove "art", "eve" and "nat" named entities, as performance on them will probably be not comparable to the other named entities. """

entities_to_remove = ["B-art", "I-art", "B-eve", "I-eve", "B-nat", "I-nat"]
data = data[~data.Tag.isin(entities_to_remove)]
data.head()
data.count()


Sentence #      47920
Word          1047063
POS           1047063
Tag           1047063
dtype: int64

In [ ]:
"""We create 2 dictionaries for NE: one that maps individual tags to indices, and one that maps indices to their individual tags. This is necessary in order to create the labels (as computers work with numbers = indices, rather than words = tags) - see further in this notebook."""

labels_to_ids = {k: v for v, k in enumerate(data.Tag.unique())}
ids_to_labels = {v: k for v, k in enumerate(data.Tag.unique())}
print(labels_to_ids)
print(ids_to_labels)

"""We create 2 dictionaries for POS: one that maps individual tags to indices, and one that maps indices to their individual tags. This is necessary in order to create the labels (as computers work with numbers = indices, rather than words = tags) - see further in this notebook."""

POS_labels_to_ids = {k: v for v, k in enumerate(data.POS.unique())}
POS_ids_to_labels = {v: k for v, k in enumerate(data.POS.unique())}
print(POS_labels_to_ids)
print(POS_ids_to_labels)


{'O': 0, 'B-geo': 1, 'B-gpe': 2, 'B-per': 3, 'I-geo': 4, 'B-org': 5, 'I-org': 6, 'B-tim': 7, 'I-per': 8, 'I-gpe': 9, 'I-tim': 10}
{0: 'O', 1: 'B-geo', 2: 'B-gpe', 3: 'B-per', 4: 'I-geo', 5: 'B-org', 6: 'I-org', 7: 'B-tim', 8: 'I-per', 9: 'I-gpe', 10: 'I-tim'}
{'NNS': 0, 'IN': 1, 'VBP': 2, 'VBN': 3, 'NNP': 4, 'TO': 5, 'VB': 6, 'DT': 7, 'NN': 8, 'CC': 9, 'JJ': 10, '.': 11, 'VBD': 12, 'WP': 13, '``': 14, 'CD': 15, 'PRP': 16, 'VBZ': 17, 'POS': 18, 'VBG': 19, 'RB': 20, ',': 21, 'WRB': 22, 'PRP$': 23, 'MD': 24, 'WDT': 25, 'JJR': 26, ':': 27, 'JJS': 28, 'WP$': 29, 'RP': 30, 'PDT': 31, 'NNPS': 32, 'EX': 33, 'RBS': 34, 'LRB': 35, 'RRB': 36, '$': 37, 'RBR': 38, ';': 39, 'UH': 40, 'FW': 41}
{0: 'NNS', 1: 'IN', 2: 'VBP', 3: 'VBN', 4: 'NNP', 5: 'TO', 6: 'VB', 7: 'DT', 8: 'NN', 9: 'CC', 10: 'JJ', 11: '.', 12: 'VBD', 13: 'WP', 14: '``', 15: 'CD', 16: 'PRP', 17: 'VBZ', 18: 'POS', 19: 'VBG', 20: 'RB', 21: ',', 22: 'WRB', 23: 'PRP$', 24: 'MD', 25: 'WDT', 26: 'JJR', 27: ':', 28: 'JJS', 29: 'WP$', 30: 'RP

In [ ]:
# count NE tag
print("Number of NE tags: {}".format(len(data.Tag.unique()))) # 17个
frequencies_NE = data.Tag.value_counts()
frequencies_NE
Ner_Tag = list(data.Tag.unique())
Ner_Number = [i for i in range(len(Ner_Tag))]
Ner = list(zip(Ner_Tag,Ner_Number))
print(Ner)


Number of NE tags: 11
[('O', 0), ('B-geo', 1), ('B-gpe', 2), ('B-per', 3), ('I-geo', 4), ('B-org', 5), ('I-org', 6), ('B-tim', 7), ('I-per', 8), ('I-gpe', 9), ('I-tim', 10)]


In [ ]:
# count POS tag
print("Number of POS tags: {}".format(len(data.POS.unique()))) # 42个
frequencies_POS = data.POS.value_counts()
frequencies_POS
POS_Tag = list(data.POS.unique())
POS_Number = [i for i in range(len(POS_Tag))]
POS = list(zip(POS_Tag,POS_Number))
print(POS)
print(len(POS_Tag))


Number of POS tags: 42
[('NNS', 0), ('IN', 1), ('VBP', 2), ('VBN', 3), ('NNP', 4), ('TO', 5), ('VB', 6), ('DT', 7), ('NN', 8), ('CC', 9), ('JJ', 10), ('.', 11), ('VBD', 12), ('WP', 13), ('``', 14), ('CD', 15), ('PRP', 16), ('VBZ', 17), ('POS', 18), ('VBG', 19), ('RB', 20), (',', 21), ('WRB', 22), ('PRP$', 23), ('MD', 24), ('WDT', 25), ('JJR', 26), (':', 27), ('JJS', 28), ('WP$', 29), ('RP', 30), ('PDT', 31), ('NNPS', 32), ('EX', 33), ('RBS', 34), ('LRB', 35), ('RRB', 36), ('$', 37), ('RBR', 38), (';', 39), ('UH', 40), ('FW', 41)]
42


In [ ]:
# pandas has a very handy "forward fill" function to fill missing values based on the last upper non-nan value
data = data.fillna(method='ffill')
data.head()


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [ ]:
# let's create a new column called "sentence" which groups the words by sentence 
data['sentence'] = data[['Sentence #','Word','Tag', 'POS']].groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(x))
# let's also create a new column called "word_labels" which groups the tags by sentence 
data['word_labels'] = data[['Sentence #','Word','Tag', 'POS']].groupby(['Sentence #'])['Tag'].transform(lambda x: ','.join(x))
data['word_labels_POS'] = data[['Sentence #','Word','Tag', 'POS']].groupby(['Sentence #'])['POS'].transform(lambda x: '_,_,_'.join(x))
data.head()


,Sentence #,Word,POS,Tag,sentence,word_labels,word_labels_POS
0,Sentence: 1,Thousands,NNS,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-...","NNS_,_,_IN_,_,_NNS_,_,_VBP_,_,_VBN_,_,_IN_,_,_..."
1,Sentence: 1,of,IN,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-...","NNS_,_,_IN_,_,_NNS_,_,_VBP_,_,_VBN_,_,_IN_,_,_..."
2,Sentence: 1,demonstrators,NNS,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-...","NNS_,_,_IN_,_,_NNS_,_,_VBP_,_,_VBN_,_,_IN_,_,_..."
3,Sentence: 1,have,VBP,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-...","NNS_,_,_IN_,_,_NNS_,_,_VBP_,_,_VBN_,_,_IN_,_,_..."
4,Sentence: 1,marched,VBN,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-...","NNS_,_,_IN_,_,_NNS_,_,_VBP_,_,_VBN_,_,_IN_,_,_..."


In [ ]:
"""Let's only keep the "sentence" and "word_labels" and "word_labels_POS" columns, and drop duplicates:"""

data = data[["sentence", "word_labels", 'word_labels_POS']].drop_duplicates().reset_index(drop=True)
data.head()

len(data)
"""Let's verify that a random sentence and its corresponding tags are correct:"""

print(data.iloc[41].sentence)
print(data.iloc[41].word_labels)
print(data.iloc[41].word_labels_POS)


Bedfordshire police said Tuesday that Omar Khayam was arrested in Bedford for breaching the conditions of his parole .
B-gpe,O,O,B-tim,O,B-per,I-per,O,O,O,B-geo,O,O,O,O,O,O,O,O
NNP_,_,_NNS_,_,_VBD_,_,_NNP_,_,_IN_,_,_NNP_,_,_NNP_,_,_VBD_,_,_VBN_,_,_IN_,_,_NNP_,_,_IN_,_,_VBG_,_,_DT_,_,_NNS_,_,_IN_,_,_PRP$_,_,_NN_,_,_.


In [ ]:
train_df, validate_df, test_df = \
              np.split(data.sample(frac=1, random_state=42), 
                       [int(.85*len(data)), int(.925*len(data))])
# train_df['word_labels'][33891]

# train_df.index= range(len(train_df))
# validate_df.index= range(len(validate_df))
# test_df.index= range(len(test_df))

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

data_combine_dict = {'train':train_df, 'validation':validate_df, 'test':test_df}


In [ ]:
class Preprocess_Data(Dataset):
  def __init__(self, dataset, tokenizer, max_len): #usage -> train, validation, test
        self.len = len(dataset)
        self.data = dataset
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):

        # step 1: get the sentence and word labels 
        # print(str(index))
        sentence = self.data.sentence[index].strip().split()
        # print(str(sentence))
        word_labels = self.data.word_labels[index].split(",")
        # print(str(word_labels))
        pos_labels = self.data.word_labels_POS[index].split("_,_,_")
        # print(str(pos_labels))
        # print('error place 2')
        
        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                              is_split_into_words=True,
                              return_offsets_mapping=True,  #Set to True to return (char_start, char_end) for each token (default False)
                              padding='max_length', 
                              truncation=True, 
                              max_length=self.max_len)
        
        
        # step 3: create token labels only for first word pieces of each tokenized word

        labels = [labels_to_ids[label] for label in word_labels]
        pos = [POS_labels_to_ids[label_POS] for label_POS in pos_labels]

        # print('error place 3')
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        encoded_pos = np.ones(len(encoding["offset_mapping"]), dtype=int) * len(POS_Tag)
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]): #training_set[i]["input_ids"]
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            encoded_pos[idx] = pos[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)

        # step 5: pos
        item['pos_ids'] = torch.as_tensor(encoded_pos)

        return item

  def __len__(self):
        return self.len        


In [ ]:
dataset = data_combine_dict

training_set = Preprocess_Data(train_df, tokenizer, MAX_LEN)
validation_set = Preprocess_Data(validate_df, tokenizer, MAX_LEN)
testing_set = Preprocess_Data(test_df, tokenizer, MAX_LEN)
print(len(training_set),len(validation_set),len(testing_set))


40439 3568 3569


In [ ]:
i = 1
for token,token_type_id,number,pos,mask,label in zip(tokenizer.convert_ids_to_tokens(training_set[i]["input_ids"]), training_set[i]["token_type_ids"],training_set[i]["input_ids"],training_set[i]["pos_ids"],training_set[i]["attention_mask"],training_set[i]["labels"]):
  print('{0:13}  {1:10}  {2:10}  {3:10}  {4:10}  {5}'.format(token, token_type_id, number, pos, mask, label))


[CLS]                   0         101          42           1  -100
A                       0         138           7           1  0
video                   0        1888           8           1  0
released                0        1308           3           1  0
to                      0        1106           5           1  0
the                     0        1103           7           1  0
Arabic                  0        4944           4           1  1
news                    0        2371           8           1  0
channel                 0        3094           8           1  0
Al                      0        2586           4           1  0
-                       0         118          42           1  -100
J                       0         147          42           1  -100
##az                    0       10961          42           1  -100
##eera                  0       27472          42           1  -100
in                      0        1107           1           1  0
November  

In [ ]:
# Define the Dataloader
training_loader = DataLoader(training_set, batch_size = TRAIN_BATCH_SIZE, shuffle=True,num_workers=0)
validation_loader = DataLoader(validation_set,batch_size = TRAIN_BATCH_SIZE, shuffle=True,num_workers=0)
testing_loader = DataLoader(testing_set,batch_size = TEST_BATCH_SIZE, shuffle=True,num_workers=0)

print(len(training_loader),len(validation_loader),len(testing_loader))


10110 892 1785


# **3. Define the Model**

### 1) Train the Model

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Dec 11 02:52:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
EPOCHS = 4
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

### Redesign the model

In [ ]:
from transformers.models.bert.modeling_bert import BertSelfAttention,BertSelfOutput,BertAttention,BertIntermediate,BertOutput,BertLayer,BertEncoder,BertPooler,BertPredictionHeadTransform,BertLMPredictionHead,BertOnlyMLMHead,BertOnlyNSPHead,BertPreTrainingHeads,BertPreTrainedModel,BertForPreTrainingOutput
from packaging import version
from transformers.file_utils import (
    ModelOutput,
    add_code_sample_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    replace_return_docstrings,
)
from transformers.modeling_outputs import BaseModelOutputWithPoolingAndCrossAttentions,TokenClassifierOutput
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

class BertEmbeddings1(nn.Module):
    """Construct the embeddings from word, position and token_type embeddings."""

    def __init__(self, config):
        super().__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)

        #self.pos_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
        self.pos_embeddings = nn.Embedding(len(POS_Tag)+1,config.hidden_size)    

        # self.LayerNorm is not snake-cased to stick with TensorFlow model variable name and be able to load
        # any TensorFlow checkpoint file
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # position_ids (1, len position emb) is contiguous in memory and exported when serialized
        self.position_embedding_type = getattr(config, "position_embedding_type", "absolute")
        self.register_buffer("position_ids", torch.arange(config.max_position_embeddings).expand((1, -1)))
        if version.parse(torch.__version__) > version.parse("1.6.0"):
            self.register_buffer(
                "token_type_ids",
                torch.zeros(self.position_ids.size(), dtype=torch.long, device=self.position_ids.device),
                persistent=False,
            )

    def forward(
        self, input_ids=None, pos_ids=None, token_type_ids=None, position_ids=None, inputs_embeds=None, past_key_values_length=0
    ):
        if input_ids is not None:
            input_shape = input_ids.size()
        else:
            input_shape = inputs_embeds.size()[:-1]

        seq_length = input_shape[1]

        if position_ids is None:
            position_ids = self.position_ids[:, past_key_values_length : seq_length + past_key_values_length]

        # Setting the token_type_ids to the registered buffer in constructor where it is all zeros, which usually occurs
        # when its auto-generated, registered buffer helps users when tracing the model without passing token_type_ids, solves
        # issue #5664
        if token_type_ids is None:
            if hasattr(self, "token_type_ids"):
                buffered_token_type_ids = self.token_type_ids[:, :seq_length]
                buffered_token_type_ids_expanded = buffered_token_type_ids.expand(input_shape[0], seq_length)
                token_type_ids = buffered_token_type_ids_expanded
            else:
                token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=self.position_ids.device)

        if inputs_embeds is None:
            inputs_embeds = self.word_embeddings(input_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)
        pos_embeddings = self.pos_embeddings(pos_ids)

        embeddings = inputs_embeds + token_type_embeddings + pos_embeddings
        if self.position_embedding_type == "absolute":
            position_embeddings = self.position_embeddings(position_ids)
            embeddings += position_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

class BertModel1(BertPreTrainedModel):
    """

    The model can behave as an encoder (with only self-attention) as well as a decoder, in which case a layer of
    cross-attention is added between the self-attention layers, following the architecture described in `Attention is
    all you need <https://arxiv.org/abs/1706.03762>`__ by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit,
    Llion Jones, Aidan N. Gomez, Lukasz Kaiser and Illia Polosukhin.

    To behave as an decoder the model needs to be initialized with the :obj:`is_decoder` argument of the configuration
    set to :obj:`True`. To be used in a Seq2Seq model, the model needs to initialized with both :obj:`is_decoder`
    argument and :obj:`add_cross_attention` set to :obj:`True`; an :obj:`encoder_hidden_states` is then expected as an
    input to the forward pass.
    """

    def __init__(self, config, add_pooling_layer=True):
        super().__init__(config)
        self.config = config

        self.embeddings = BertEmbeddings1(config)
        self.encoder = BertEncoder(config)

        self.pooler = BertPooler(config) if add_pooling_layer else None

        self.init_weights()

    def get_input_embeddings(self):
        return self.embeddings.word_embeddings

    def set_input_embeddings(self, value):
        self.embeddings.word_embeddings = value

    def _prune_heads(self, heads_to_prune):
        """
        Prunes heads of the model. heads_to_prune: dict of {layer_num: list of heads to prune in this layer} See base
        class PreTrainedModel
        """
        for layer, heads in heads_to_prune.items():
            self.encoder.layer[layer].attention.prune_heads(heads)

    #@add_start_docstrings_to_model_forward(BERT_INPUTS_DOCSTRING.format("batch_size, sequence_length"))
    #@add_code_sample_docstrings(
    #    processor_class=_TOKENIZER_FOR_DOC,
    #    checkpoint=_CHECKPOINT_FOR_DOC,
    #   output_type=BaseModelOutputWithPoolingAndCrossAttentions,
    #    config_class=_CONFIG_FOR_DOC,
    #)
    def forward(
        self,
        input_ids=None,
        pos_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_values=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        encoder_hidden_states  (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, sequence_length, hidden_size)`, `optional`):
            Sequence of hidden-states at the output of the last layer of the encoder. Used in the cross-attention if
            the model is configured as a decoder.
        encoder_attention_mask (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Mask to avoid performing attention on the padding token indices of the encoder input. This mask is used in
            the cross-attention if the model is configured as a decoder. Mask values selected in ``[0, 1]``:

            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.
        past_key_values (:obj:`tuple(tuple(torch.FloatTensor))` of length :obj:`config.n_layers` with each tuple having 4 tensors of shape :obj:`(batch_size, num_heads, sequence_length - 1, embed_size_per_head)`):
            Contains precomputed key and value hidden states of the attention blocks. Can be used to speed up decoding.

            If :obj:`past_key_values` are used, the user can optionally input only the last :obj:`decoder_input_ids`
            (those that don't have their past key value states given to this model) of shape :obj:`(batch_size, 1)`
            instead of all :obj:`decoder_input_ids` of shape :obj:`(batch_size, sequence_length)`.
        use_cache (:obj:`bool`, `optional`):
            If set to :obj:`True`, :obj:`past_key_values` key value states are returned and can be used to speed up
            decoding (see :obj:`past_key_values`).
        """
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if self.config.is_decoder:
            use_cache = use_cache if use_cache is not None else self.config.use_cache
        else:
            use_cache = False

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input_shape = input_ids.size()
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        batch_size, seq_length = input_shape
        device = input_ids.device if input_ids is not None else inputs_embeds.device

        # past_key_values_length
        past_key_values_length = past_key_values[0][0].shape[2] if past_key_values is not None else 0

        if attention_mask is None:
            attention_mask = torch.ones(((batch_size, seq_length + past_key_values_length)), device=device)

        if token_type_ids is None:
            if hasattr(self.embeddings, "token_type_ids"):
                buffered_token_type_ids = self.embeddings.token_type_ids[:, :seq_length]
                buffered_token_type_ids_expanded = buffered_token_type_ids.expand(batch_size, seq_length)
                token_type_ids = buffered_token_type_ids_expanded
            else:
                token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)
        
        # if pos_ids is None:
        #   pos_ids = torch.zeros(input_shape,dtype=torch.long,device=device)
        # else:
        #   pos_ids = pos_ids.to(device)

        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape, device)

        # If a 2D or 3D attention mask is provided for the cross-attention
        # we need to make broadcastable to [batch_size, num_heads, seq_length, seq_length]
        if self.config.is_decoder and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
            encoder_extended_attention_mask = self.invert_attention_mask(encoder_attention_mask)
        else:
            encoder_extended_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embedding_output = self.embeddings(
            input_ids=input_ids,
            pos_ids=pos_ids,
            position_ids=position_ids,
            token_type_ids=token_type_ids,
            inputs_embeds=inputs_embeds,
            past_key_values_length=past_key_values_length,
        )
        encoder_outputs = self.encoder(
            embedding_output,
            attention_mask=extended_attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_extended_attention_mask,
            past_key_values=past_key_values,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output) if self.pooler is not None else None

        if not return_dict:
            return (sequence_output, pooled_output) + encoder_outputs[1:]

        return BaseModelOutputWithPoolingAndCrossAttentions(
            last_hidden_state=sequence_output,
            pooler_output=pooled_output,
            past_key_values=encoder_outputs.past_key_values,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
            cross_attentions=encoder_outputs.cross_attentions,
        )

#@add_start_docstrings(
#    """
#    Bert Model with two heads on top as done during the pretraining: a `masked language modeling` head and a `next
#    sentence prediction (classification)` head.
#    """,
#    BERT_START_DOCSTRING,
#)

class BertForTokenClassification1(BertPreTrainedModel):

    _keys_to_ignore_on_load_unexpected = [r"pooler"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel1(config, add_pooling_layer=False)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    #@add_start_docstrings_to_model_forward(BERT_INPUTS_DOCSTRING.format("batch_size, sequence_length"))
    #@add_code_sample_docstrings(
    #    processor_class=_TOKENIZER_FOR_DOC,
    #    checkpoint=_CHECKPOINT_FOR_DOC,
    #    output_type=TokenClassifierOutput,
    #    config_class=_CONFIG_FOR_DOC,
    #)
    def forward(
        self,
        input_ids=None,
        pos_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Labels for computing the token classification loss. Indices should be in ``[0, ..., config.num_labels -
            1]``.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            pos_ids=pos_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]

        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            # Only keep active parts of the loss
            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.num_labels)
                active_labels = torch.where(
                    active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
                )
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )



#@add_start_docstrings(
#    """
#    Bert Model with a span classification head on top for extractive question-answering tasks like SQuAD (a linear
#    layers on top of the hidden-states output to compute `span start logits` and `span end logits`).
#    """,
#    BERT_START_DOCSTRING,
#)

In [ ]:
device

'cuda'

### Train the model

In [ ]:
# Define the model by just BertForTokenClassification
model = BertForTokenClassification1.from_pretrained('bert-base-cased', num_labels=len(Ner_Tag))
model.to(device)

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
import time
import os

In [ ]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):

        # if idx >200:
        #   break
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)
        pos = batch['pos_ids'].to(device,dtype = torch.long)

        outputs = model(input_ids=ids, pos_ids=pos,attention_mask=mask, labels=labels)
        loss = outputs[0]
        tr_logits = outputs[1]
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
            if idx!=0:
              print("--- %s seconds ---" % (time.time() - start_time))
            start_time = time.time()    
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

    # --------------------------------------------------------------------------------------------------------------------
    # After the completion of each training epoch
    # measure our performance on validation set.

    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(validation_loader):

            # if idx >200:
            #   break
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            pos = batch['pos_ids'].to(device,dtype = torch.long)
            
            # outputs= model(input_ids=ids, attention_mask=mask, labels=labels)
            outputs = model(input_ids=ids, pos_ids=pos,attention_mask=mask, labels=labels)
            loss = outputs[0]
            eval_logits = outputs[1]
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

In [ ]:
len(training_loader),len(validation_loader),len(testing_loader)

(10110, 892, 1785)

In [ ]:
from seqeval.metrics import classification_report
New_NerDict = dict((v,k) for k,v in dict(Ner).items())
New_NerDict

{0: 'O',
 1: 'B-geo',
 2: 'B-gpe',
 3: 'B-per',
 4: 'I-geo',
 5: 'B-org',
 6: 'I-org',
 7: 'B-tim',
 8: 'I-per',
 9: 'I-gpe',
 10: 'I-tim'}

In [ ]:
EPOCHS = 3
for epoch in range(EPOCHS):

    directory = "/content/drive/MyDrive/NLP/Final Project/New Dataset/Bank/POS_Embedding_layer/Cased/Model_"+str(epoch+1)

    if not os.path.exists(directory):
        os.makedirs(directory)


    print(f"Training epoch: {epoch + 1}")
    train(epoch)
    labels, predictions = valid(model, testing_loader)

    labels_value = [[New_NerDict[i.item()] for i in labels]]
    pred_value = [[New_NerDict[i.item()] for i in predictions]]

    print(classification_report(labels_value, pred_value))

    tokenizer.save_vocabulary(directory)
    # save the model weights and its configuration file
    model.save_pretrained(directory)
    print('All files saved')

Output hidden; open in https://colab.research.google.com to view.

### 2) Evaluate the Model

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            pos = batch['pos_ids'].to(device,dtype = torch.long)

            outputs = model(input_ids=ids, pos_ids=pos,attention_mask=mask, labels=labels)
            loss = outputs[0]
            eval_logits = outputs[1]

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
                if idx!=0:
                  print("--- %s seconds ---" % (time.time() - start_time))
                start_time = time.time()
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy


    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Testing Loss: {eval_loss}")
    print(f"Testing Accuracy: {eval_accuracy}")

    return eval_labels, eval_preds


In [ ]:
len(testing_loader)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/NLP/Final Project/Code/Baseline/Baseline Saved Model/pytorch_model.bin'))

In [ ]:
labels, predictions = valid(model, testing_loader)

In [ ]:
New_NerDict = dict((v,k) for k,v in dict(Ner).items())
New_NerDict

In [ ]:
labels_value = [[New_NerDict[i.item()] for i in labels]]
pred_value = [[New_NerDict[i.item()] for i in predictions]]
print(labels_value)
print(pred_value)

In [ ]:
from seqeval.metrics import classification_report

print(classification_report(labels_value, pred_value))

### 3) Save Model

In [ ]:
import os

directory = "./model"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')

In [ ]:
#torch.save(model, 'model.pth')

#torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')